In [2]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit

In [3]:
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

In [4]:
formtemplatescategoriesy=formtemplatescategories.copy()
formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
formtemplates1=formtemplates.copy()
formtemplates1=formtemplates1.rename({"_id":"id"},axis=1)
formtemplates1=formtemplates1.rename({"_featuredCategory":"_id"},axis=1)
df1_melt = formtemplates1.assign(_categories=formtemplates1._categories.str.split(","))
formtemplates1=df1_melt._categories.apply(pd.Series) \
    .merge(formtemplates1, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_id","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates1,how="inner",on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates1=formtemplates1.rename({"_id":"_featuredCategory"},axis=1)


In [5]:
formtemplates1

,id,_title,_slug,_description,_featuredCategory,_language,_categories
0,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,9134504740,en,4f854c5a365599ea2c000009
1,10243036686,Quick Quote,quick-quote-private-2,\nInvestment Property Loans\n\n\nNo tax or per...,NaN,en,4f854c5a365599ea2c00001c
2,10403313912,Facilities request form,calendar-facilities-request-form,This form allows you to manage in a very effic...,9127600941,en,4f854c5a365599ea2c000017
3,10471410826,Ghislaine_Contact,ghislainecontact,contact,NaN,nl,4f854c5a365599ea2c000009
4,10613649813,Ghislaine_Producten,ghislaineproducten,producten,NaN,nl,4f854c5a365599ea2c000009
...,...,...,...,...,...,...,...
11449155,93420783566161,Inside Sales Call Report Form,inside-sales-call-report-form,Here is a daily call report form that is craft...,4f855d5aceae3f4968000000,en,NaN
11449156,93420809987167,Daily Leads Report Form,daily-leads-report-form,Here is a daily leads report form for inside s...,4f855d5aceae3f4968000000,en,NaN
11449157,93420845091153,Inside Sales Daily Emails Report Form,inside-sales-daily-emails-report-form,Here is a daily email report form that is craf...,4f855d5aceae3f4968000000,en,NaN
11449158,93420896655468,Ø§Ø³ØªÙ…Ø§Ø±Ø© Ø§Ù„ØªØ³Ø¬ÙŠÙ„ ÙÙŠ Ø¯ÙˆØ±Ø© Øª...,ltsgyl-fy-dwrh-tdrybyh,Ù†Ù…ÙˆØ°Ø¬ ØªØ³Ø¬ÙŠÙ„ ÙÙŠ Ø¯ÙˆØ±Ø© ØªØ¯Ø±ÙŠØ¨...,9248123036,ar,NaN


In [6]:
formtemplates1=formtemplates1.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates1,how="inner",on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates1=formtemplates1.rename({"_id":"_categories"},axis=1)
formtemplates1=formtemplates1.rename({"id":"_id"},axis=1)
formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
formtemplates2=formtemplates2.drop_duplicates()
formtemplates3=formtemplates3.drop_duplicates()

In [7]:
formtemplates2

,_featuredCategory,name,slug,description,pageTitle,metaKeywords,id,_title,_slug,_description,_language,_categories
0,4f854c5a365599ea2c000001,Contact Forms,contact-form,Contact forms are an essential part of any bus...,Contact Forms,Contact Forms,200122438355042,Get In Touch Contact Form,get-in-touch-contact-form,<p>A Get In Touch Contact Form is used to coll...,en,
1,4f854c5a365599ea2c000001,Contact Forms,contact-form,Contact forms are an essential part of any bus...,Contact Forms,Contact Forms,200205206936143,Bridal Makeup Contact Form,bridal-makeup-contact-form,A bridal makeup contact form gathering bride's...,en,4f854c5a365599ea2c000017
2,4f854c5a365599ea2c000001,Contact Forms,contact-form,Contact forms are an essential part of any bus...,Contact Forms,Contact Forms,200351465180042,Popup Contact Form,popup-contact-form,A Popup Contact Form is used to collect contac...,en,4f854c5a365599ea2c000009
3,4f854c5a365599ea2c000001,Contact Forms,contact-form,Contact forms are an essential part of any bus...,Contact Forms,Contact Forms,200544196772357,Contact Form Template in Italian,contact-form-template-in-italian,Contact form provides you with the submitters'...,it,
4,4f854c5a365599ea2c000001,Contact Forms,contact-form,Contact forms are an essential part of any bus...,Contact Forms,Contact Forms,200548992928873,Enquiry Form Template,enquiry-form-template,Enquiry form template asks your submitters on ...,en,
...,...,...,...,...,...,...,...,...,...,...,...,...
798460,5c3f238e8a021f5c833d4331,Photography Forms,photography,Looking for online photography forms? Jotform ...,NaN,Photography Forms,92622840008351,Fecha TÃ©cnica de ColecciÃ³n de FotografÃ­as,fecha-tecnica-de-coleccion-de-fotografias,<p>Si organiza exposiciones de arte o fotograf...,es,NaN
798471,5c3f238e8a021f5c833d4331,Photography Forms,photography,Looking for online photography forms? Jotform ...,NaN,Photography Forms,71571279433864,InformaciÃ³n de contacto y servicio de fotogra...,informacion-de-contacto-y-servicio-de-fotografia,<p>Dirigida a fotÃ³grafos profesionales especi...,es,9135076442
798495,5c3f238e8a021f5c833d4331,Photography Forms,photography,Looking for online photography forms? Jotform ...,NaN,Photography Forms,71571279433864,InformaciÃ³n de contacto y servicio de fotogra...,informacion-de-contacto-y-servicio-de-fotografia,<p>Dirigida a fotÃ³grafos profesionales especi...,es,4f854c5a365599ea2c000015
798519,5c3f238e8a021f5c833d4331,Photography Forms,photography,Looking for online photography forms? Jotform ...,NaN,Photography Forms,71571279433864,InformaciÃ³n de contacto y servicio de fotogra...,informacion-de-contacto-y-servicio-de-fotografia,<p>Dirigida a fotÃ³grafos profesionales especi...,es,


In [8]:
formtemplates3

,_categories,name,slug,description,pageTitle,metaKeywords,id,_title,_slug,_description,_featuredCategory,_language
0,1013514931,Pharmacy Forms,pharmacy-forms,Free online forms for your pharmacy. Whether y...,NaN,Pharmacy Forms,203651425373149,COVID-19 Vaccination E-registration Form,covid-19-vaccination-e-registration-jan-15-202...,<p>The COVID-19 Vaccination E-registration For...,NaN,en
1,1013514931,Pharmacy Forms,pharmacy-forms,Free online forms for your pharmacy. Whether y...,NaN,Pharmacy Forms,210063102726037,CDC COVID-19 Vaccination Program Provider Agre...,cdc-covid-19-vaccination-program-provider-agre...,The CDC (Centers of Disease Control and Preven...,1082515029,en
2,1013514931,Pharmacy Forms,pharmacy-forms,Free online forms for your pharmacy. Whether y...,NaN,Pharmacy Forms,210075599735969,Prescription Request Form,prescription-request-form,<p>Prescription Request Form is a document or ...,4f854c5a365599ea2c00000e,en
3,1013514931,Pharmacy Forms,pharmacy-forms,Free online forms for your pharmacy. Whether y...,NaN,Pharmacy Forms,210114594077048,CDC COVID-19 Vaccine Immunization Records,cdc-covid-19-vaccine-immunization-records,CDC form,1082515029,en
4,1013514931,Pharmacy Forms,pharmacy-forms,Free online forms for your pharmacy. Whether y...,NaN,Pharmacy Forms,210255742988060,COVID-19 Vaccine Administration Form,covid-19-vaccine-administration-form,COVID-19 Vaccine Administration Form allows pa...,1082515029,en
...,...,...,...,...,...,...,...,...,...,...,...,...
86132,9283064443,Quizzes,quiz-template,Create interactive quizzes for your school or ...,NaN,"quiz templates, quiz samples",90095431342148,Alternative Education Site Application,private-211,AE Form,NaN,aa
86133,9283064443,Quizzes,quiz-template,Create interactive quizzes for your school or ...,NaN,"quiz templates, quiz samples",93245428785165,Office365 Contacts,office365-contacts,share,NaN,aa
86134,9283064443,Quizzes,quiz-template,Create interactive quizzes for your school or ...,NaN,"quiz templates, quiz samples",200860054262446,Broti Order Form,broti-order-form-private-1,TEST,NaN,aa
86135,9283064443,Quizzes,quiz-template,Create interactive quizzes for your school or ...,NaN,"quiz templates, quiz samples",201346124481043,FICHA DE INSCRIPCIÃ“N FÃšTBOL TEMPORADA 20/21 ...,ficha-de-inscripcion-futbol-temporada-20-21-cp...,INCRIPCIÃ“N CLUB,NaN,aa


In [9]:
formcat=pd.concat([formtemplates2,formtemplates3]).drop_duplicates()
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_id']]
formandclone=formandclone.loc[:,[
'_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    '_featuredCategory',
    'formID',
    'form_type',
    'source',
    'date']]


In [10]:
formcat

,name,description,metaKeywords,_categories,_id
0,Contact Forms,Contact forms are an essential part of any bus...,Contact Forms,,200122438355042
1,Contact Forms,Contact forms are an essential part of any bus...,Contact Forms,4f854c5a365599ea2c000017,200205206936143
2,Contact Forms,Contact forms are an essential part of any bus...,Contact Forms,4f854c5a365599ea2c000009,200351465180042
3,Contact Forms,Contact forms are an essential part of any bus...,Contact Forms,,200544196772357
4,Contact Forms,Contact forms are an essential part of any bus...,Contact Forms,,200548992928873
...,...,...,...,...,...
86132,Quizzes,Create interactive quizzes for your school or ...,"quiz templates, quiz samples",9283064443,90095431342148
86133,Quizzes,Create interactive quizzes for your school or ...,"quiz templates, quiz samples",9283064443,93245428785165
86134,Quizzes,Create interactive quizzes for your school or ...,"quiz templates, quiz samples",9283064443,200860054262446
86135,Quizzes,Create interactive quizzes for your school or ...,"quiz templates, quiz samples",9283064443,201346124481043


In [11]:
formandclone

,_id,_title,_slug,_description,_language,_featuredCategory,formID,form_type,source,date
0,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,en,9134504740,200010384803440,OLD,TEMPLATES,2020-01-02 00:28:52
1,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,en,9134504740,200022509812442,OLD,TEMPLATES,2020-01-03 04:37:50
2,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,en,9134504740,200038449888162,OLD,TEMPLATES,2020-01-04 21:15:24
3,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,en,9134504740,200053141418440,OLD,TEMPLATES,2020-01-06 08:43:05
4,10175020727,Simple Employee Application,job-application-3,Here's a nice little employment application th...,en,9134504740,200075837085456,OLD,TEMPLATES,2020-01-08 13:29:25
...,...,...,...,...,...,...,...,...,...,...
8961565,93420896655468,Ø§Ø³ØªÙ…Ø§Ø±Ø© Ø§Ù„ØªØ³Ø¬ÙŠÙ„ ÙÙŠ Ø¯ÙˆØ±Ø© Øª...,ltsgyl-fy-dwrh-tdrybyh,Ù†Ù…ÙˆØ°Ø¬ ØªØ³Ø¬ÙŠÙ„ ÙÙŠ Ø¯ÙˆØ±Ø© ØªØ¯Ø±ÙŠØ¨...,ar,9248123036,231988531281563,OLD,TEMPLATES,2023-07-18 21:55:55
8961566,93421610431142,Inside Sales Email Report Form,inside-sales-email-report-form,"<p class=\""description-area\"" style=\""padding:...",en,4f855d5aceae3f4968000000,211001012553533,OLD,TEMPLATES,2021-04-11 01:31:59
8961567,93421610431142,Inside Sales Email Report Form,inside-sales-email-report-form,"<p class=\""description-area\"" style=\""padding:...",en,4f855d5aceae3f4968000000,211762983269467,OLD,TEMPLATES,2021-06-26 08:07:49
8961568,93421610431142,Inside Sales Email Report Form,inside-sales-email-report-form,"<p class=\""description-area\"" style=\""padding:...",en,4f855d5aceae3f4968000000,211763462788466,OLD,TEMPLATES,2021-06-26 08:07:49


In [ ]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from statsmodels.tsa.seasonal import seasonal_decompose

from plotly.subplots import make_subplots
import plotly.graph_objects as go


class decompose:
    def __init__(self):
        self.formtemplatesclones0=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
        self.formtemplatescategories0=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
        self.formtemplates0=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")
    def sd(self,yaxis):
        self.formtemplatesclones=self.formtemplatesclones0.copy()
        self.formtemplatescategories=self.formtemplatescategories0.copy()
        self.formtemplates=self.formtemplates0.copy()
        self.yaxis=yaxis                    
        self.formtemplatescategoriesy=self.formtemplatescategories[self.formtemplatescategories["metaKeywords"]==self.yaxis].copy()
        self.formtemplatesclones=self.formtemplatesclones.rename({"templateID":"_id"},axis=1)
        formandclone=pd.merge(self.formtemplates,self.formtemplatesclones,how="inner",on="_id")
        self.formtemplates=self.formtemplates.rename({"_id":"id"},axis=1)
        self.formtemplates=self.formtemplates.rename({"_featuredCategory":"_id"},axis=1)
        self.formtemplates2=pd.merge(self.formtemplatescategoriesy,self.formtemplates,how="inner",on="_id")
        self.formtemplates2=self.formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
        self.formtemplates=self.formtemplates.rename({"_id":"_featuredCategory"},axis=1)
        df_melt = self.formtemplates.assign(_categories=self.formtemplates._categories.str.split(","))
        self.formtemplates=df_melt._categories.apply(pd.Series) \
            .merge(self.formtemplates, right_index=True, left_index=True) \
            .drop(["_categories"], axis=1) \
            .melt(id_vars=['id',"_title","_slug","_description","_featuredCategory","_language"], value_name="_categories") \
            .drop("variable", axis=1) 
        self.formtemplates=self.formtemplates.rename({"_categories":"_id"},axis=1)
        self.formtemplates3=pd.merge(self.formtemplatescategoriesy,self.formtemplates,how="inner",on="_id")
        self.formtemplates3=self.formtemplates3.rename({"_id":"_categories"},axis=1)
        self.formtemplates=self.formtemplates.rename({"_id":"_categories"},axis=1)
        self.formtemplates=self.formtemplates.rename({"id":"_id"},axis=1)
        self.formtemplatesclones=self.formtemplatesclones.rename({"_id":"templateID"},axis=1)
        formcat=pd.concat([self.formtemplates2,self.formtemplates3])
        formcat=formcat.rename({"id":"_id"},axis=1)
        formcat=formcat.loc[:,[
        'name',
        'description',
        'metaKeywords',
        '_categories',
        '_id']]
        formandclone=formandclone.loc[:,[
        '_id',
        '_title',
        '_slug',
        '_description',
        '_language',
        '_featuredCategory',
        'formID',
        'form_type',
        'source',
        'date']]
        formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
        formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
        formclonecategnonnany.index=formclonecategnonnany.date
        formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
        formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
        formclonecategnonnany["day"]=formclonecategnonnany.index.dayofweek
        formclonecategnonnany["week"]=formclonecategnonnany.index.week
        formclonecategnonnany["dayofyear"]=formclonecategnonnany.index.dayofyear
        formclonecategnonnany["year"]=formclonecategnonnany.index.year
        formclonecategnonnany["quarter"]=formclonecategnonnany.index.quarter
        formclonecategnonnany["month"]=formclonecategnonnany.index.month
        formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()
        df=psql.sqldf("select count(_id),date,year,dayofyear,week,day,quarter,month from formclonecategnonnany  group by date")
        df.index=df.date.tolist()
        df.index=pd.to_datetime(df.index)
        df.index=df.index.to_period("D")
        df.index=df.index.to_timestamp()
        df.index=df.index.strftime('%d-%m-%Y')
        df["date"]=pd.to_datetime(df["date"])
        df=df.rename({"count(_id)":"y"},axis=1)
        df=df.rename({"date":"ds"},axis=1)
        df=df.loc[:,["y","ds"]]
        df=df.resample('7D',on="ds").sum()
        decomposeresult2=seasonal_decompose(df["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
        self.season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
        self.season.index=pd.to_datetime(self.season.index,format="%d-%m-%Y")
        self.season=self.season.fillna(0)
        self.season=self.season.sort_index()
        self.season.index=self.season.index.to_period("D")
        self.season.index=self.season.index.to_timestamp()
        self.season.index=self.season.index.strftime('%d-%m-%Y')
        self.season.index=pd.to_datetime(self.season.index,format='%d-%m-%Y')
        fig=make_subplots(rows=4, cols=1,
                        subplot_titles=(self.yaxis, "Trend", "Seasonal_yearly","resid"))
        fig.add_trace(
            go.Scatter(x=self.season.index,y=self.season.obs),row=1,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=self.season.index,y=self.season.trend),row=2,col=1
        )
        fig.add_trace(
            go.Scatter(x=self.season.index,y=self.season.seasonal),row=3,col=1
            
        )
        fig.add_trace(
            go.Scatter(x=self.season.index,y=self.season.resid),row=4,col=1
            
        )


        fig.update_layout(height=1000, width=950)
        return fig


